In [5]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {'scroll': True,})

%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.abspath("."))

from viewer import ThreeJsViewer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Frame and Transformation

<img src="https://gramaziokohler.github.io/compas_fab/latest/_images/02_coordinate_frames.jpg"  />

## Frame

A frame is defined by a base point and two orthonormal base vectors (xaxis, yaxis), which specify the normal (zaxis). 
It describes location and orientation in a (right-handed) cartesian coordinate system.

<div align="middle"><img src='images/frame.svg' style='height:350px' /></div>

In [1]:
"""There are several ways to construct a `Frame`.
"""
from compas.geometry import Point
from compas.geometry import Vector
from compas.geometry import Frame
from compas.geometry import Plane

# Frame autocorrects axes to be orthonormal
F = Frame(Point(1, 0, 0), Vector(-0.45, 0.1, 0.3), Vector(1, 0, 0))

print(F)

F = Frame([1, 0, 0], [-0.45, 0.1, 0.3], [1, 0, 0])

F = Frame.from_points([1, 1, 1], [2, 3, 6], [6, 3, 0])
F = Frame.from_plane(Plane([0, 0, 0], [0.5, 0.2, 0.1]))
F = Frame.from_euler_angles([0.5, 1., 0.2])
F = Frame.worldXY()

print(F)

Frame(Point(1.000, 0.000, 0.000), Vector(-0.818, 0.182, 0.545), Vector(0.575, 0.259, 0.776))
Frame(Point(0.000, 0.000, 0.000), Vector(1.000, 0.000, 0.000), Vector(0.000, 1.000, 0.000))


### Frame as a cartesian coordinate system

<img src='images/point_in_frame.svg' style='height:350px' />

In [2]:
"""Example: 'point in frame'
"""
point = Point(146.00, 150.00, 161.50)
xaxis = Vector(0.9767, 0.0010, -0.214)
yaxis = Vector(0.1002, 0.8818, 0.4609)

# coordinate system F
F = Frame(point, xaxis, yaxis)

# point in F (local coordinates)
P = Point(35., 35., 35.)

# point in global (world) coordinates
P_ = F.to_world_coords(P)
print(P_)

# check
P2 = F.to_local_coords(P_)
print(P2)

Point(190.314, 164.389, 200.285)
Point(35.000, 35.000, 35.000)


The simple example above shows how to use a frame as a coordinate system: Starting from a point `P` in the local (user-defined, relative) coordinate system of frame `F`, i.e. its position is relative to the origin and orientation of `F`, we want to get the position `P_` of `P` in the global (world, absolute) coordinate system.

### Frame in frame

<img src='images/frame_in_frame_point.svg' style='height:450px' />

In [3]:
"""Example: 'frame in frame'
# TODO update graphic!
"""
from compas.geometry import * 

# coordinate system F0
point = Point(146.00, 150.00, 161.50)
xaxis = Vector(0.9767, 0.0010, -0.214)
yaxis = Vector(0.1002, 0.8818, 0.4609)
F0 = Frame(point, xaxis, yaxis)

# frame F in F0 (local coordinates)
point = Point(35., 35., 35.)
xaxis = Vector(0.604, 0.430, 0.671)
yaxis = Vector(-0.631, 0.772, 0.074)
F = Frame(point, xaxis, yaxis)

# frame in global (world) coordinate system
F_wcf = F0.to_world_coords(F)
print(F_wcf)

# check
F2 = F0.to_local_coords(F_wcf)
print(F2)
print(F == F2)

Frame(Point(190.314, 164.389, 200.285), Vector(0.760, 0.063, 0.647), Vector(-0.526, 0.645, 0.554))
Frame(Point(35.000, 35.000, 35.000), Vector(0.604, 0.430, 0.671), Vector(-0.631, 0.772, 0.074))
True


### Example

Bring a box from the world coordinate system into another coordinate system.

In [4]:
"""Example: Bring a box from the world coordinate system into another coordinate system.
"""
from compas.geometry import Frame
from compas.geometry import Box

# Box in the world coordinate system
frame = Frame([1, 0, 0], [-0.45, 0.1, 0.3], [1, 0, 0])
width, length, height = 1, 1, 1
box = Box(frame, width, length, height)

# Frame F representing a coordinate system
F = Frame([2, 2, 2], [0.978, 0.010, -0.210], [0.090, 0.882, 0.463])

# Represent box frame in frame F and construct new box
box_frame_transformed = F.to_world_coords(box.frame)
box_transformed = Box(box_frame_transformed, width, length, height)
print("Box frame transformed:", box_transformed.frame)

Box frame transformed: Frame(Point(2.978, 2.010, 1.790), Vector(-0.680, -0.105, 0.726), Vector(0.733, -0.132, 0.668))


In [6]:
from viewer import ThreeJsViewer
viewer = ThreeJsViewer()
viewer.draw_frame(Frame.worldXY(), line_width=2)
viewer.draw_box(box)
viewer.draw_frame(F, line_width=2)
viewer.draw_box(box_transformed, color='#3FB59D')
camera_position=[5.0, -5.0, 2.0]
viewer.show(camera_position)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(intensity=0.5, position=(0.0, 0.0, 1.…

### Further information

* https://en.wikipedia.org/wiki/Frame_of_reference
* https://en.wikipedia.org/wiki/Cartesian_coordinate_system

## Transformation

Transformations refer to operations such as moving, rotating, and scaling objects. They are stored using 4x4 transformation matrices. 

Most transformations preserve the parallel relationship among the parts of the geometry. For example collinear points 
remain collinear after the transformation. Also points on one plane stay coplanar after transformation. This type of
transformation is called an *affine transformation* and concerns transformations such as `Rotation`, `Translation`, 
`Scale`, `Reflection`, `Shear` and orthogonal and parallel `Projection`. Only perspective `Projection` is not *affine*.

<img src='images/transformations2D.svg' style='width:800px' />

A transformation matrix looks like this:

\begin{equation*}
\mathbf{T} = \begin{bmatrix}
a_{11} & a_{12} & a_{13} & a_{14}\\
a_{21} & a_{22} & a_{23} & a_{24}\\
a_{31} & a_{32} & a_{33} & a_{34}\\
a_{41} & a_{42} & a_{43} & a_{44}\\
\end{bmatrix}
\end{equation*}

But for different transformations, different coefficients are used:

The `Rotation` just uses the upper left 3x3 coefficients,

\begin{equation*}
\mathbf{T} = \begin{bmatrix}
a_{11} & a_{12} & a_{13} & 0\\
a_{21} & a_{22} & a_{23} & 0\\
a_{31} & a_{32} & a_{33} & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix}
\end{equation*}

the `Translation` just uses the first 3 coefficients of the 4th column,

\begin{equation*}
\mathbf{T} = \begin{bmatrix}
1 & 0 & 0 & a_{14}\\
0 & 1 & 0 & a_{24}\\
0 & 0 & 1 & a_{34}\\
0 & 0 & 0 & 1\\
\end{bmatrix}
\end{equation*}

and `Scale` uses only the first 3 values of the diagonal.

\begin{equation*}
\mathbf{T} = \begin{bmatrix}
a_{11} & 0 & 0 & 0\\
0 & a_{22} & 0 & 0\\
0 & 0 & a_{33} & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix}
\end{equation*}

In [13]:
from compas.geometry import *

axis, angle = [0.2, 0.4, 0.1], 0.3
R = Rotation.from_axis_and_angle(axis, angle)
print("Rotation:\n", R)

translation_vector = [5, 3, 1]
T = Translation(translation_vector)
print("Translation:\n", T)

scale_factors = [0.1, 0.3, 0.4]
S = Scale(scale_factors)
print("Scale:\n", S)

point, normal = [0.3, 0.2, 1], [0.3, 0.1, 1]
R = Reflection(point, normal) # TODO: should take a plane!!
print("Reflection:\n", R)

point, normal = [0, 0, 0], [0, 0, 1]
perspective = [1, 1, 0]
P = Projection.perspective(point, normal, perspective)
print("Perspective projection:\n", R)

angle, direction = 0.1, [0.1, 0.2, 0.3]
point, normal = [4, 3, 1], [-0.11, 0.31, -0.17]
S = Shear(angle, direction, point, normal)
print("Shear:\n", S)

Rotation:
 [[    0.9638,   -0.0475,    0.2622,    0.0000],
 [    0.0815,    0.9894,   -0.1205,    0.0000],
 [   -0.2537,    0.1375,    0.9575,    0.0000],
 [    0.0000,    0.0000,    0.0000,    1.0000]]

Translation:
 [[    1.0000,    0.0000,    0.0000,    5.0000],
 [    0.0000,    1.0000,    0.0000,    3.0000],
 [    0.0000,    0.0000,    1.0000,    1.0000],
 [    0.0000,    0.0000,    0.0000,    1.0000]]

Scale:
 [[    0.1000,    0.0000,    0.0000,    0.0000],
 [    0.0000,    0.3000,    0.0000,    0.0000],
 [    0.0000,    0.0000,    0.4000,    0.0000],
 [    0.0000,    0.0000,    0.0000,    1.0000]]

Reflection:
 [[    0.8364,   -0.0545,   -0.5455,    0.6055],
 [   -0.0545,    0.9818,   -0.1818,    0.2018],
 [   -0.5455,   -0.1818,   -0.8182,    2.0182],
 [    0.0000,    0.0000,    0.0000,    1.0000]]

Perspective projection:
 [[    0.8364,   -0.0545,   -0.5455,    0.6055],
 [   -0.0545,    0.9818,   -0.1818,    0.2018],
 [   -0.5455,   -0.1818,   -0.8182,    2.0182],
 [    0.0000,

### Inverse transformation

The inverse transformation $T^{-1}$ is calculated through inverting the transformation matrix $T$. 

$T \times T^{-1} = I$, with $I$ as identity matrix. 

In [7]:
"""Example: Transform a point and invert the transformation
"""
from compas.geometry import *
from math import pi

p = Point(3, 4, 5)
T = Rotation.from_axis_and_angle([2, 2, 2], pi/4)

# transform Point p with T
p.transform(T)
print(p) # transformed point

# create inverse Transformation to T
Tinv = T.inverse()

# transform Point p with inverse Transformation 
p.transform(Tinv)

# check if p has the same values as in the beginning
print(p) # == (3, 4, 5)

# what is the result of multiplying T with Tinv?
print(T * Tinv)

Point(3.701, 3.184, 5.115)
Point(3.000, 4.000, 5.000)
[[    1.0000,    0.0000,    0.0000,    0.0000],
 [    0.0000,    1.0000,    0.0000,    0.0000],
 [    0.0000,    0.0000,    1.0000,    0.0000],
 [    0.0000,    0.0000,    0.0000,    1.0000]]



### Concatenation

The concatenation of several `Transformations` are simple matrix multiplications.
But matrices are not commutative ($\mathbf{A}\times\mathbf{B} \neq \mathbf{B}\times\mathbf{A}$), 
so it is important to consider the order of multiplication.

There are 2 ways to concatenate transformation matrices:

<table style="font-size:100%">
<tr style="background: none;">
<td style="text-align: right">
Pre-multiplication:<br/>
Post-multiplication:<br/>
</td>
<td style="text-align: left">
$\mathbf{C}=\mathbf{B}\times\mathbf{A}$<br/>
$\mathbf{C}=\mathbf{A}\times\mathbf{B}$<br/>
</td>
<td style="text-align: left">
(apply $\mathbf{B}$ to the left of $\mathbf{A}$)<br/>
(apply $\mathbf{B}$ to the right of $\mathbf{A}$)<br/>
</td>
</tr>
</table>

Which one you choose depends on what you want to do...

### Pre-multiplication 

Think of transformations with respect to the global (world) coordinate system.

<div align="center"><img src="images/pre-multiplication.svg" width="800" /></div>

## Pre-multiplication

If you transform an object first with transformation $\mathbf{A}$, then with transformation $\mathbf{B}$, 
followed by transformation $\mathbf{C}$, you get the same result as transforming the object with only 
one transformation $\mathbf{M}$ which is calculated by $\mathbf{M}=\mathbf{C}\times\mathbf{B}\times\mathbf{A}$.
(Transformations applied from right to left.)

In [8]:
"""Example: Pre-multiply transformations
"""
from compas.geometry import *

p = Point(1, 1, 1)

translation = [1, 2, 3]
A = Translation(translation) # create Translation
axis, angle = [-0.8, 0.35, 0.5], 2.2
B = Rotation.from_axis_and_angle(axis, angle) # create Rotation
scale_factors = [0.1, 0.3, 0.4]
C = Scale(scale_factors) # create Scale

# Transform one by one
p1 = p.transformed(A)
p1.transform(B)
p1.transform(C)

# Transform with only one concatenated matrix
p2 = p.transformed(C * B * A)

print(p1)
print(p2)
allclose(p1, p2)

Point(-0.308, 0.722, -1.483)
Point(-0.308, 0.722, -1.483)


True

### Post-multiplication 

Think of transformations as transforming the local coordinate frame.

<div align="center"><img src="images/post-multiplication.svg" width="800" /></div>

In [15]:
"""Example: pre-multiplication"""
import math
R = Rotation.from_axis_and_angle([0, 0, 1], math.radians(30))
T = Translation([2, 0, 0])
S = Scale([0.5] * 3)
C = S * T * R
C

[[    0.4330,   -0.2500,    0.0000,    1.0000],
 [    0.2500,    0.4330,    0.0000,    0.0000],
 [    0.0000,    0.0000,    0.5000,    0.0000],
 [    0.0000,    0.0000,    0.0000,    1.0000]]

In [16]:
"""Example: post-multiplication"""
import math
R = Rotation.from_axis_and_angle([0, 0, 1], math.radians(30))
T = Translation([2, 0, 0])
S = Scale([0.5] * 3)
C = R * T * S
C

[[    0.4330,   -0.2500,    0.0000,    1.7321],
 [    0.2500,    0.4330,    0.0000,    1.0000],
 [    0.0000,    0.0000,    0.5000,    0.0000],
 [    0.0000,    0.0000,    0.0000,    1.0000]]

In [9]:
"""Example: Decompose transformations
"""
from compas.geometry import *

scale_factors = [0.1, 0.3, 0.4]
A = Scale(scale_factors) # create Scale
axis, angle = [-0.8, 0.35, 0.5], 2.2
B = Rotation.from_axis_and_angle(axis, angle) # create Rotation
translation = [1, 2, 3]
C = Translation(translation) # create Translation

# Concatenate transformations
M = C * B * A

# A matrix can also be decomposed into it's components of Scale, 
# Shear, Rotation, Translation and Perspective
Sc, Sh, R, T, P = M.decomposed()
# Check
print(A == Sc)
print(B == R)
print(C == T)
print(P * T * R * Sh * Sc == M)

True
True
True
True


### Question

To transform a point, you simply multiply the point with the transformation matrix. But a transformation matrix is 4x4 
and vectors and points are 3x1, so how can they be multiplied?

### Matrix multiplication

If $\mathbf{A}$ is an $m \times n$ matrix and $\mathbf{B}$ is an $n \times p$ matrix, the matrix product $\mathbf{C} = \mathbf{A}\mathbf{B}$ is defined to be a $m \times p$ matrix.

<table style="font-size: 100%;">
<tr style="background: none;">
<td style="text-align: left;">
\begin{equation*}
{\overset {4\times 2{\text{ matrix}}}{\begin{bmatrix}a_{11}&a_{12}\\a_{21}&a_{22}\\a_{31}&a_{32}\\a_{41}&a_{42}\\\end{bmatrix}}}
{\overset {2\times 3{\text{ matrix}}}{\begin{bmatrix}b_{11}&b_{12}&b_{13}\\b_{21}&b_{22}&b_{23}\\\end{bmatrix}}}=
{\overset {4\times 3{\text{ matrix}}}{\begin{bmatrix}x_{11}&x_{12}&x_{13}\\x_{21}&x_{22}&x_{23}\\x_{31}&x_{32}&x_{33}\\x_{41}&x_{42}&x_{43}\\\end{bmatrix}}} 
\end{equation*}
<br />
<br />
$x_{12}=a_{11}b_{12}+a_{12}b_{22}\\
x_{33}=a_{31}b_{13}+a_{32}b_{23}$
</td>
<td>
<img width="300" src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Matrix_multiplication_diagram_2.svg/470px-Matrix_multiplication_diagram_2.svg.png" />
</td>
</tr>
</table>

https://en.wikipedia.org/wiki/Matrix_multiplication

### Homogenisation

To transform points and vectors, i.e. multiply them with the transformation matrix, we need to homogenize them first.
This means representing a 3-vector (x, y, z) as a 4-vector (x, y, z, 1) for points, or (x, y, z, 0) for vectors.

Points:
\begin{equation*}
\begin{bmatrix}x'\\y'\\z'\\1\end{bmatrix} = 
\begin{bmatrix}
a_{11} & a_{12} & a_{13} & a_{14}\\
a_{21} & a_{22} & a_{23} & a_{24}\\
a_{31} & a_{32} & a_{33} & a_{34}\\
a_{41} & a_{42} & a_{43} & a_{44}\\
\end{bmatrix}
\times
\begin{bmatrix}x\\y\\z\\1\end{bmatrix}
\end{equation*}

Vectors:
\begin{equation*}
\begin{bmatrix}x'\\y'\\z'\\0\end{bmatrix} = 
\begin{bmatrix}
a_{11} & a_{12} & a_{13} & a_{14}\\
a_{21} & a_{22} & a_{23} & a_{24}\\
a_{31} & a_{32} & a_{33} & a_{34}\\
a_{41} & a_{42} & a_{43} & a_{44}\\
\end{bmatrix}
\times
\begin{bmatrix}x\\y\\z\\0\end{bmatrix}
\end{equation*}

**NOTE**:
<p style="background-color: yellow;">
That is one of the reasons for distinguishing between points and vectors!
</p>

In [10]:
"""Example: transform point and vector
"""
# create Transformation
R = Rotation.from_axis_and_angle([-0.248, -0.786, -0.566], 2.78, point=[1.0, 0.0, 0.0])

# apply Transformation to Point
p = Point(1, 1, 1)
p.transform(T)

# apply Transformation to Vector
v = Vector(1, 1, 1)
v.transform(T)

# print them both
print(p)
print(v)

Point(2.000, 3.000, 4.000)
Vector(1.000, 1.000, 1.000)


## Transformation of multiple points

If you need to transform multiple points it is better to do this with the methods `transform_points` or `transform_points_numpy`
than applying the transformation on each point. (likewise `transform_vectors` or `transform_vectors_numpy`)

In [42]:
import time
import compas
from compas.datastructures import Mesh
from compas.geometry import transform_points
from compas.geometry import transform_points_numpy

# load mesh
mesh = Mesh.from_ply(compas.get('bunny.ply'))
v, f = mesh.to_vertices_and_faces()
print("The mesh has {} vertices.".format(len(v)))

# create Transformation
T = Rotation.from_axis_and_angle([-0.248, -0.786, -0.566], 2.78, point=[1.0, 0.0, 0.0])

# transform points with transform_points
t0 = time.time()
transform_points(v, T)
print("transfrom_points takes {:.4f} seconds.".format(time.time() - t0))

# transform points with transform_points_numpy
t0 = time.time()
transform_points_numpy(v, T)
print("transfrom_points_numpy takes {:.4f} seconds.".format(time.time() - t0))

The mesh has 35947 vertices.
transfrom_points takes 0.3279 seconds.
transfrom_points_numpy takes 0.0435 seconds.


**NOTE**:
<p style="background-color: yellow;">
If you want to apply several transformations on a big mesh, it is faster to multiply transformations first 
and then apply only one transformation to mesh.
</p>

## Frame and Transformation

<img src='images/frame_transformation.svg' style='height:450px' />

### Difference between change-basis transformation and transformation between frames.

`Transformation.change_basis(f1, f2)` and `Transformation.from_frame_to_frame(f1, f2)`

A change-basis transformation allows to remap geometry of one coordinate system in another, i.e. represent the same coordinates in 2 different frames,
whereas the Transformation between 2 frames allows to transform geometry from one coordinate system into the other one.

In [11]:
"""This example computes a transformation between two frames F1 and F2.
"""
from compas.geometry import Frame
from compas.geometry import Transformation

F1 = Frame([2, 2, 2], [0.12, 0.58, 0.81], [-0.80, 0.53, -0.26])
F2 = Frame([1, 1, 1], [0.68, 0.68, 0.27], [-0.67, 0.73, -0.15])

T = Transformation.from_frame_to_frame(F1, F2)
print(T)

# now transform F1 with T
F2 == F1.transformed(T)

[[    0.7933,    0.2220,    0.5668,   -2.1644],
 [   -0.4588,    0.8301,    0.3169,   -0.3765],
 [   -0.4002,   -0.5115,    0.7604,    1.3025],
 [    0.0000,    0.0000,    0.0000,    1.0000]]



True

In [12]:
"""This example computes a change-basis transformation between two frames F1 and F2.
"""
from compas.geometry import Frame
from compas.geometry import Transformation

F1 = Frame([2, 2, 2], [0.12, 0.58, 0.81], [-0.80, 0.53, -0.26])
F2 = Frame([1, 1, 1], [0.68, 0.68, 0.27], [-0.67, 0.73, -0.15])

T = Transformation.change_basis(F1, F2)
print(T)

# F1 local = worldXY F1 in F2
Point(0, 0, 0).transformed(T) == F2.to_local_coords(F1.point)

[[    0.6931,   -0.2561,   -0.6738,    1.6319],
 [    0.2201,    0.9653,   -0.1406,   -0.0903],
 [    0.6864,   -0.0508,    0.7254,    0.5734],
 [    0.0000,    0.0000,    0.0000,    1.0000]]



True

In [13]:
"""Example: Bring a box from the world coordinate system into another coordinate system.
"""
from compas.geometry import Frame
from compas.geometry import Transformation
from compas.geometry import Box

# Box in the world coordinate system
frame = Frame([1, 0, 0], [-0.45, 0.1, 0.3], [1, 0, 0])
width, length, height = 1, 1, 1
box = Box(frame, width, length, height)

# Frame F representing a coordinate system
F = Frame([2, 2, 2], [0.978, 0.010, -0.210], [0.090, 0.882, 0.463])

# Get transformation between frames and apply transformation on box.
T = Transformation.from_frame_to_frame(Frame.worldXY(), F)
box_transformed = box.transformed(T)
print("Box frame transformed", box_transformed.frame)

Box frame transformed Frame(Point(2.978, 2.010, 1.790), Vector(-0.680, -0.105, 0.726), Vector(0.733, -0.132, 0.668))


In [14]:
viewer = ThreeJsViewer()
viewer.draw_frame(Frame.worldXY())
viewer.draw_box(box)
viewer.draw_frame(F)
viewer.draw_box(box_transformed, color="#00aaff")
camera_position=[5.0, -5.0, 2.0]
viewer.show(camera_position)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(intensity=0.5, position=(0.0, 0.0, 1.…

### Difference between `transform()` and `transformed()`


* Make a copy of box and transform it (don't change box):
        box_transformed = box.transformed(T)
* Transform box (returns `None`):
        box.transform(T)

**NOTE**:
<p style="background-color: yellow;">
Depending on the size of the object, copying takes time. So consider carefully if you really need a copy.
</p>

## Rotation and orientation

A `Rotation` is a circular movement of an object around a point of rotation. A three-dimensional object can always be rotated around an infinite number of imaginary lines called rotation axes.


In [26]:
"""There are several ways to construct a `Rotation`.
"""
import math
from compas.geometry import Rotation

R = Rotation.from_axis_and_angle([1, 0, 0], math.radians(30))
R = Rotation.from_axis_and_angle([1, 0, 0], math.radians(30), point=[1, 0, 0])
R = Rotation.from_basis_vectors([0.68, 0.68, 0.27], [-0.67, 0.73, -0.15])
R = Rotation.from_frame(Frame([1, 1, 1], [0.68, 0.68, 0.27], [-0.67, 0.73, -0.15]))
R = Rotation.from_axis_angle_vector([-0.043, -0.254, 0.617])
R = Rotation.from_quaternion([0.945, -0.021, -0.125, 0.303])
R = Rotation.from_euler_angles([1.4, 0.5, 2.3], static=True, axes='xyz')

print(R)

[[   -0.5847,   -0.4415,    0.6806,    0.0000],
 [    0.6544,    0.2391,    0.7173,    0.0000],
 [   -0.4794,    0.8648,    0.1492,    0.0000],
 [    0.0000,    0.0000,    0.0000,    1.0000]]



In [28]:
import math
from compas.geometry import Sphere

viewer = ThreeJsViewer()
sphere = viewer.draw_sphere(Sphere((0,2,0), 0.5))

times = []
transformations = []

for i in range(0, 360, 1):
    transformations.append(Rotation.from_axis_and_angle([0,0,1], math.radians(i)))
    times.append(i * 0.01)

sphere_action = viewer.create_action(sphere, transformations, times)

In [29]:
camera_position=[0.0, 0.0, 10.0]
viewer.show(camera_position=camera_position, action=sphere_action)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(intensity=0.5, matrixWorldNeedsUpdate…

AnimationAction(clip=AnimationClip(duration=3.5899999141693115, tracks=(VectorKeyframeTrack(name='.position', …

### Euler angles

The Euler angles are three angles introduced by Leonhard Euler to describe the orientation of a rigid body with respect
to a fixed coordinate system.

The three elemental rotations may be 
* extrinsic (`static=True`, rotations about the axes xyz of the original coordinate system, which is assumed to remain motionless), or
* intrinsic (`static=False`, rotations about the axes of the rotating coordinate system XYZ, solidary with the moving body, which changes its orientation after each elemental rotation).

Euler angles are typically denoted as α, β, γ. Different authors use different sets of rotation axes to 
define Euler angles (`axes='xyz'`,`axes='xyx'`, ...) or different names for the same angles.
In flight dynamics, the principal rotations described with Euler angles are known as *pitch*, *roll* and *yaw*.

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/Euler.png/543px-Euler.png' style='height:450px' />

In [30]:
from compas.geometry import *

box = Box(Frame([0,0,0], [1,0,0], [0,1,0]), 1., 1., 1.)
 
alpha, beta, gamma = 0.3, 0.5, 0.7
xaxis, yaxis, zaxis = [1, 0, 0], [0, 1, 0], [0, 0, 1]

Rx = Rotation.from_axis_and_angle(xaxis, alpha)
Ry = Rotation.from_axis_and_angle(yaxis, beta)
Rz = Rotation.from_axis_and_angle(zaxis, gamma)

In [31]:
viewer = ThreeJsViewer()
jbox = viewer.draw_box(box)
frame = viewer.draw_frame(box.frame)

times = [0, 3, 5, 8, 10, 13, 17]
transformations = [Transformation()]
transformations += [Rx] * 2
transformations += [Rx * Ry] * 2
transformations += [Rx * Ry * Rz] * 2

box_action = viewer.create_action(jbox, transformations, times)
frame_action = viewer.create_action(frame, transformations, times)

In [32]:
viewer.show()

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(intensity=0.5, matrixWorldNeedsUpdate…

In [33]:
box_action

AnimationAction(clip=AnimationClip(duration=17.0, tracks=(VectorKeyframeTrack(name='.position', times=array([ …

In [34]:
frame_action

AnimationAction(clip=AnimationClip(duration=17.0, tracks=(VectorKeyframeTrack(name='.position', times=array([ …

In [35]:
"""Example: Rotations from euler angles, rotate an object based on 3 euler angles.
"""

import compas
from compas.geometry import Frame
from compas.geometry import Rotation
from compas.datastructures import Mesh
from compas.datastructures import mesh_transform

# euler angles
alpha, beta, gamma = -0.156, -0.274, 0.785
static, axes = True, 'xyz'

# Version 1: Create Rotation from angles
R1 = Rotation.from_euler_angles([alpha, beta, gamma], static, axes)

# Version 2: Concatenate 3 Rotations
xaxis, yaxis, zaxis = [1, 0, 0], [0, 1, 0], [0, 0, 1]
Rx = Rotation.from_axis_and_angle(xaxis, alpha)
Ry = Rotation.from_axis_and_angle(yaxis, beta)
Rz = Rotation.from_axis_and_angle(zaxis, gamma)
 
if static: # check difference between pre- and post-concatenation!
    R2 = Rz * Ry * Rx
else:
    R2 = Rx * Ry * Rz

# Check
print(R1 == R2)

True


### Axis-angle representation

The axis–angle representation of a rotation parameterizes a rotation in a three-dimensional Euclidean space by two 
quantities: a unit vector $\mathbf{e}$ indicating the direction of an axis of rotation, and an angle $θ$ describing 
the magnitude of the rotation about the axis.

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/7/7b/Angle_axis_vector.svg/300px-Angle_axis_vector.svg.png' style='height:250px' />

In [36]:
"""Example: Create a rotation from and axis and an angle.
"""
from compas.geometry import *
aav = Vector(-0.043, -0.254, 0.617)
angle, axis = aav.unitized(), aav.length
print(angle, axis)
R = Rotation.from_axis_angle_vector(aav)
axis, angle = R.axis_and_angle
print(axis, angle)

Vector(-0.064, -0.380, 0.923) 0.6686209688605346
Vector(-0.064, -0.380, 0.923) 0.6686209688605343


### Unit Quaternion

In mathematics, the quaternions are a number system that extends the complex numbers. 

Quaternions are generally represented in the form:

${\displaystyle a+b\ \mathbf{i} +c\ \mathbf{j} +d\ \mathbf{k} }$

where $a$, $b$, $c$, and $d$ are real numbers, and $\mathbf{i}$, $\mathbf{j}$, and $\mathbf{k}$ are the fundamental quaternion units.

Unit quaternions, also known as versors, provide a convenient mathematical notation for representing orientations and 
rotations of objects in three dimensions. Compared to Euler angles they are simpler to compose and compared to 
rotation matrices they are more compact, more numerically stable, and more efficient. 


In [37]:
from compas.geometry import Rotation
from compas.geometry import Quaternion

q = Quaternion(0.918958, -0.020197, -0.151477, 0.363544)
print(q.is_unit)
R = Rotation.from_quaternion(q)
print(R.quaternion == q)

True
True


In [38]:
"""Example: Different Robot vendors use different conventions to describe TCP orientation."""

from compas.geometry import Point
from compas.geometry import Vector
from compas.geometry import Frame

point = Point(0.0, 0.0, 63.0)
xaxis = Vector(0.68, 0.68, 0.27)
yaxis = Vector(-0.67, 0.73, -0.15)
F = Frame(point, xaxis, yaxis)

print(F.quaternion) # ABB
print(F.euler_angles(static=False, axes='xyz')) # Staubli
print(F.euler_angles(static=False, axes='zyx')) # KUKA
print(F.axis_angle_vector) # UR

Quaternion(0.916525, -0.019350, -0.155237, 0.368117)
[0.08268917200390957, -0.3034379323245168, 0.7764953222906119]
[0.7853981633974483, -0.27371608012579823, -0.1561844248311574]
Vector(-0.040, -0.319, 0.757)


### Assignment Task 1

Project the box corner coordinates of a (rotated and translated) box onto the xy-plane. You can use either 
orthogonal, parallel or perspective Projection. In the end visualize the edges of the projected box corners
(tip: Mesh will help you there).


<div align="center"><br><img src="images/assignment1_1.jpg" width="600" /></div>

### Further information

* https://en.wikipedia.org/wiki/Transformation_matrix
* https://en.wikipedia.org/wiki/Euler_angles
* https://en.wikipedia.org/wiki/Axis%E2%80%93angle_representation
* https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation